## Установка зависимостей

In [ ]:
# !pip install -q diffusers transformers accelerate omegaconf invisible-watermark

In [ ]:
# выполните, если получаете ошибку "cannot import name 'CLIPImageProcessor' from 'transformers'" при импорте SD XL

# !pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install -q ipywidgets

## Stable Diffusion XL generation

HuggingFace: https://huggingface.co/docs/diffusers/using-diffusers/sdxl

Про другие модели можно почитать [здесь](https://neerc.ifmo.ru/wiki/index.php?title=%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D1%83)

Подробнее про задачу [здесь](https://habr.com/ru/articles/713076/)

In [ ]:
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0.safetensors", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

refiner = StableDiffusionXLImg2ImgPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/blob/main/sd_xl_refiner_1.0.safetensors", torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
).to("cuda")

### Text-to-image*

*StableDiffusion также умеет решать задачи **Image-to-image**, **inpainting** (подробнее расссказано [здесь](https://neerc.ifmo.ru/wiki/index.php?title=%D0%92%D0%BF%D0%B8%D1%81%D1%8B%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%87%D0%B0%D1%81%D1%82%D0%B8_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F))

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

prompt = input('Введите Ваш запрос: ')
image = pipeline_text2image(prompt=prompt).images[0]
image

## BLIP annotation (Image Caption)

Подробнее о модели CLIP можно прочитать [здесь](https://huggingface.co/Salesforce/blip-image-captioning-large)

Увидеть пример использования другой модели (CLIP) можно [здесь](https://github.com/Technolog796/image_captioning)

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# Условное Image Caption
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

# Image captioning без ограничений
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))